# Data Visualization for Mildew Detection in Cherry Leaves

## Objectives
* Understand the visual differences between healthy and powdery mildew-infected cherry leaves.
* Analyze the distribution of image sizes and ensure the dataset is balanced.
* Prepare visual content that can be used for the dashboard or further analysis.

## Inputs
* Preprocessed dataset with images categorized into healthy and powdery mildew classes.

## Outputs
* Visualizations including sample images, average images, and variability images for each class.
* Any intermediate files or plots saved for future reference or use in presentations.

## Additional Comments
* Visualization is key to understanding the data and guiding the model development process.

---

# Set Data Directory

## Import libraries